# Setup

In [1]:
import os, base64
from dotenv import load_dotenv
from openai import OpenAI
from pdf2image import convert_from_path

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from IPython.display import Markdown, display

# Perform OCR and transform to images

In [3]:
def pdf_to_images(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    imagens = convert_from_path(pdf_path)
    image_paths = []

    pdf_nome = os.path.splitext(os.path.basename(pdf_path))[0]
    for i, imagem in enumerate(imagens):
        image_path = os.path.join(output_folder, f"{pdf_nome}_pg{i+1}.jpg")
        imagem.save(image_path, "JPEG")
        image_paths.append(image_path)

    return image_paths

In [4]:
# Convertendo pdf para img
main_folder = os.path.join(os.getcwd(), "5.rag_openai")
receips_folder = os.path.join(main_folder, "receips")
output_folder = os.path.join(main_folder, "img")

pdf_path = os.path.join(receips_folder, "Southern Cookbook of Fine Recipes.pdf")
image_paths = pdf_to_images(pdf_path, output_folder)

In [5]:
client = OpenAI(api_key=openai_api_key)
model = "gpt-4o-mini"

In [6]:
# Read and encode one image
image_path = os.path.join(output_folder, "Southern Cookbook of Fine Recipes_pg9.jpg")
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
system_prompt = """
Analyse the content of this image and extract any related recipe information.
"""

In [13]:
def generate(client, system_prompt, image, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                "This is the image from the recipe page.",
                {"type": "image_url",
                 "image_url": {"url": f"data:image/jpeg;base64,{image}",
                               "detail": "high"}}
            ]}
        ],
        temperature=0
    )
    return response.choices[0].message.content

In [15]:
response = generate(client, system_prompt, image_data)
display(Markdown(response))

Here's a summary of the recipes included in the provided image:

### Chicken Gumbo
- **Ingredients**: 
  - 1 small stewing chicken
  - 2 tablespoons flour
  - 3 tablespoons butter, melted
  - 1 onion, chopped
  - 4 cups okra, sliced and chopped
  - 2 cups tomato pulp
  - Few sprigs parsley, chopped
  - Salt and pepper to taste
- **Instructions**: 
  1. Clean and cut chicken into portions.
  2. Dredge chicken in flour.
  3. Sauté in melted butter with chopped onion until browned.
  4. Season with salt and pepper.
  5. Cook slowly for about 2½ hours until chicken is tender.
  6. Add okra and parsley, cooking until okra is well done.

### Almond Chicken Soup
- **Ingredients**: 
  - ½ cup blanched almonds
  - 6 bitter almonds
  - 3 cups chicken broth
  - 1 teaspoon onion juice
  - 1 bay leaf, crushed fine
  - 3 tablespoons butter
  - 2 cups milk
  - 1 cup cream
  - Salt and pepper to taste
- **Instructions**: 
  1. Chop almonds and add to broth with seasoning.
  2. Melt butter, stir in flour, add to broth until smooth.
  3. Add milk and cream and season with salt and pepper.

### Southern Bean Soup
- **Ingredients**: 
  - 1 cup dried beans
  - 6 cups ham broth
  - 1 cup chopped celery
  - ½ onion, minced
  - 3 tablespoons butter
  - 3 tablespoons flour
  - Salt and pepper
- **Instructions**: 
  1. Cover beans with water and let stand overnight.
  2. Drain and add to ham broth.
  3. Cook with celery and onion until beans are soft.
  4. Strain broth, press beans through a sieve, and add to broth.
  5. Thicken with butter, flour, salt, and pepper.

### Additional Recipes
- **Chicken Cream Soup**
- **Okra Soup**
- **Black Bean Soup**
- **Southern Gumbo**

These recipes reflect traditional Southern cooking, featuring hearty ingredients and methods that emphasize slow cooking and robust flavors.

In [11]:
def get_gpt_response(client, system_prompt, imagem):
    response = generate(client, system_prompt, imagem)
    return [response, display(Markdown(response))]

In [12]:
# Melhorando o system prompt
system_prompt = """
Analyse the content of this image and extract any related recipe information into structure components.
Specifically, extract the recipe title, list of ingredients, step by step instructions, cousine type, dish type, any relevant tags or metadata.
The output must be in a way suited for embedding in a Retrieval Augmented Generation (RAG) system.
"""

In [14]:
# Teste
resposta = get_gpt_response(client, system_prompt, image_data)
resposta[1]

Here’s the structured recipe information extracted from the image:

### Recipe Title
Chicken Gumbo

### Cuisine Type
Southern

### Dish Type
Soup

### Ingredients
- 1 small stewing chicken
- 2 tablespoons flour, melted
- 3 tablespoons butter
- 1 onion, chopped
- 4 cups okra, sliced and chopped
- 2 cups tomato pulp
- Few sprigs parsley, chopped
- Salt and pepper to taste
- 4 cups water

### Instructions
1. Clean and dress the chicken, cutting it into serving portions.
2. Dredge the chicken lightly with flour and sauté in melted butter along with the chopped onion.
3. Once the chicken is nicely browned, add the okra, tomatoes, parsley, and water.
4. Season to taste with salt and pepper.
5. Cook slowly until the chicken is tender and the okra is well-cooked (about 2½ hours).
6. If a thinner soup is preferred, increase the quantity of water as needed.

### Relevant Tags
- Comfort Food
- Hearty
- Traditional
- Southern Cuisine

This format is suitable for embedding in a Retrieval Augmented Generation (RAG) system.

---
#### Precisa Melhorar
No resultado, não foi mostrada todas as receitas da imagem. É preciso ir testando os prompts e parâmetros.
<img src="5.rag_openai/img/Southern%20Cookbook%20of%20Fine%20Recipes_pg9.jpg" width="700"/>

Devido custo e tempo, vamos continuar com o que temos.

In [24]:
def converter_imagens_b64(imagens_path):
    imagens_b64 = []
    for imagem_path in imagens_path:
        with open(imagem_path, "rb") as arquivo_imagem:
            imagens_b64.append(base64.b64encode(arquivo_imagem.read()).decode('utf-8'))
    return imagens_b64

In [18]:
def gerar_respostas(client, system_prompt, lista_imagens_b64):
    respostas = []
    for image_data in lista_imagens_b64:
        respostas.append(generate(client, system_prompt, image_data))
    return respostas

In [26]:
def converter_gerar_tudo(client, system_prompt, imagens_path):
    imagens_b64 = converter_imagens_b64(imagens_path)
    responses = gerar_respostas(client, system_prompt, imagens_b64)
    output = []
    for i, imagem_path in enumerate(imagens_path):
        output.append(
            {"imagem_path": imagem_path, "conteudo": responses[i]}
        )
    return output

In [27]:
def filtrar_receitas(receitas):
    receitas_filtradas = []
    for receita in receitas:
        if any(keyword in receita["conteudo"].lower() for keyword in ["ingredients", "instructions", "recipe title"]):
            receitas_filtradas.append(receita)
        else:
            print(f"Ignorando receita: {receita["imagem_path"]}")
    return receitas_filtradas

In [25]:
len(converter_imagens_b64(image_paths))

52